## Imports

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

## DQN Algo Outline

In [3]:
from collections import namedtuple
import random
import time

import torch
from torch import nn
from torch import optim
import numpy as np

import gym
from src import agent, train, utils

In [4]:
random.seed(0)

## Explore

### Collect 10 random frames

In [5]:
replay_mem_size = int(1e6)
batch_size = 32
num_episodes = int(1e3)

In [6]:
def frames_to_phi(frames):
    frames = [utils.process_frame(s) 
              for s in frames]
    phi = torch.cat(frames)
    
    return phi

def get_rand_phis(k, n):
    frames = []
    env = gym.envs.make('Pong-v4')
    env.reset()
    
    for i in range(n):
        a = random.randrange(env.action_space.n)
        s_t1, r_t, done, _ = env.step(a)
        frames.append(s_t1)

        if done:
            s_t = env.reset()
    
    idxs = random.sample(range(3, n), k)
    phis = [frames_to_phi(frames[i-3:i+1]) for i in idxs]
    
    return phis

def get_frames_avg_qval(phis, agt):
    x = torch.stack(phis)
    qvals = agt.get_best_values(x)
    
    return torch.mean(qvals).item()

In [15]:
def initialize(replay_mem_size, batch_size):
    agt = agent.DQNAgent()
    replay_mem = utils.ReplayMemory(replay_mem_size, batch_size)
    obs_history = utils.ObsHistory()
    env = gym.envs.make('Pong-v4')
    train_stats = TrainingStats()
    
    return agt, replay_mem, obs_history, env, train_stats

def act_step(obs_history, agt, env):
    phi_t = obs_history.get_phi()
    a_t = agt.act(phi_t)
    s_t1, r_t, done, _ = env.step(a_t)
    
    return a_t, s_t1, r_t, done

def store_step(s_t, a_t, r_t, done, s_t1, obs_history, replay_mem):
    obs_history.store(s_t1)
    replay_mem.store(s_t, a_t, r_t, done)

def gradient_step(replay_mem, agt):
    if replay_mem.size > replay_mem.sample_size + 3:
        mini_batch = replay_mem.sample()

        agt.optimizer.zero_grad()
        loss = train.mini_batch_loss(mini_batch, agt)
        loss.backward()
        agt.optimizer.step()
        
        return loss.item()

def save_params(agt, episodes, save_path):
    torch.save({
        'model_state_dict': agt.qnet.state_dict(),
        'optimizer_state_dict': agt.optimizer.state_dict(),
        'episodes': episodes
    }, save_path)

def load_params(agt, load_path):
    checkpoint = torch.load(checkpoint_path)
    
    agt.qnet.load_state_dict(checkpoint['model_state_dict'])
    agt.opimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    return chekpoint['episodes']

def reset_episode(env, obs_history):
    s_t = env.reset()
    obs_history.reset(s_t)
    done = False
    
    return s_t, done

In [16]:
def frames_to_phi(frames):
    frames = [utils.process_frame(s) for s in frames]
    phi = torch.cat(frames)

    return phi

def get_rand_phis(k, n):
    frames = []
    env = gym.envs.make('Pong-v4')
    env.reset()

    for i in range(n):
        a = random.randrange(env.action_space.n)
        s_t1, r_t, done, _ = env.step(a)
        frames.append(s_t1)

        if done:
            s_t = env.reset()

    idxs = random.sample(range(3, n), k)
    phis = [frames_to_phi(frames[i-3:i+1]) for i in idxs]

    return phis

In [28]:
class TrainingStats:
    
    def __init__(self):
        self.ep_rewards = []
        self.ep_avg_train_losses = []
        self.steps_per_ep = []
        self.benchmark_qvals = []
        self.benchmark_frames = torch.stack(get_rand_phis(10, 10000))
        
    def store(self, agt, ep_reward, ep_steps, ep_loss, episode_num):
        self.ep_rewards.append(ep_reward)
        self.steps_per_ep.append(ep_steps)
        
        avg_ep_loss = ep_loss / ep_steps
        self.ep_avg_train_losses.append(avg_ep_loss)
        
        avg_qvals = self.get_frames_avg_qval(agt)
        self.benchmark_qvals.append(avg_qvals)
        
        print('Episode {}:'.format(episode_num))
        print('Reward: {}'.format(ep_reward))
        print('Steps: {}'.format(ep_steps))
        print('Avg loss: {:.5f}'.format(avg_ep_loss))
        print('Avg qvals: {:.5f}'.format(avg_qvals))
        print('===========================================')
        

    def get_frames_avg_qval(self, agt):
        qvals = agt.get_best_values(self.benchmark_frames)

        return torch.mean(qvals).item()

In [29]:
num_episodes = 10000

In [30]:
ep_rewards = []
ep_avg_train_losses = []
steps_per_ep = []
benchmark_qvals = []

In [31]:
agt, replay_mem, obs_history, env, train_stats = \
    initialize(replay_mem_size, batch_size)

for episode in range(num_episodes):  # loop over episodes
    s_t, done = reset_episode(env, obs_history)
    
    ep_reward = 0
    ep_train_loss = 0
    ep_steps = 0
    
    while not done:  # loop over steps in episode
        a_t, s_t1, r_t, done = act_step(obs_history, agt, env)
        store_step(s_t, a_t, r_t, done, s_t1, obs_history, replay_mem)
        
        s_t = s_t1
        
        loss_val = gradient_step(replay_mem, agt)
        
        ep_reward += r_t
        ep_steps += 1
        if loss_val is not None:
            ep_train_loss += loss_val 
    
    train_stats.store(agt, ep_reward, ep_steps, ep_train_loss, episode)
    
    if episode % 10 == 9:
        checkpoint_name = 'dqn_agt_{}.pt'.format(episode)
        save_params(agt, episode, checkpoint_name)
        print('Model saved.')
    

Episode 0:
Reward: -21.0
Steps: 1166
Avg loss: 0.01169
Avg qvals: -0.67752
Episode 1:
Reward: -21.0
Steps: 1163
Avg loss: 0.01747
Avg qvals: -1.73713
Episode 2:
Reward: -21.0
Steps: 1084
Avg loss: 0.01702
Avg qvals: -2.48010
Episode 3:
Reward: -21.0
Steps: 1005
Avg loss: 0.01802
Avg qvals: -1.26754
Episode 4:
Reward: -21.0
Steps: 1146
Avg loss: 0.01635
Avg qvals: -1.31768
Episode 5:
Reward: -19.0
Steps: 1249
Avg loss: 0.01854
Avg qvals: -1.03654
Episode 6:
Reward: -21.0
Steps: 1182
Avg loss: 0.01664
Avg qvals: -1.44374
Episode 7:
Reward: -20.0
Steps: 1199
Avg loss: 0.01664
Avg qvals: -1.61514
Episode 8:
Reward: -21.0
Steps: 1019
Avg loss: 0.01696
Avg qvals: -1.65590
Episode 9:
Reward: -21.0
Steps: 1093
Avg loss: 0.01862
Avg qvals: -2.41460
Model saved.
Episode 10:
Reward: -20.0
Steps: 1308
Avg loss: 0.01718
Avg qvals: -1.54633
Episode 11:
Reward: -21.0
Steps: 1227
Avg loss: 0.01787
Avg qvals: -1.73531
Episode 12:
Reward: -20.0
Steps: 1252
Avg loss: 0.01768
Avg qvals: -2.01598
Episode 1

Episode 68:
Reward: -21.0
Steps: 1035
Avg loss: 0.01713
Avg qvals: -1.61855
Episode 69:
Reward: -21.0
Steps: 1076
Avg loss: 0.01767
Avg qvals: -1.27528
Model saved.
Episode 70:
Reward: -21.0
Steps: 1164
Avg loss: 0.01743
Avg qvals: -1.48893
Episode 71:
Reward: -21.0
Steps: 1039
Avg loss: 0.01780
Avg qvals: -1.79250
Episode 72:
Reward: -21.0
Steps: 1061
Avg loss: 0.01779
Avg qvals: -1.55489
Episode 73:
Reward: -20.0
Steps: 1557
Avg loss: 0.01821
Avg qvals: -1.76433
Episode 74:
Reward: -21.0
Steps: 1131
Avg loss: 0.01727
Avg qvals: -1.71705
Episode 75:
Reward: -19.0
Steps: 1363
Avg loss: 0.01661
Avg qvals: -1.68427
Episode 76:
Reward: -19.0
Steps: 1348
Avg loss: 0.01803
Avg qvals: -1.59744
Episode 77:
Reward: -21.0
Steps: 1024
Avg loss: 0.01760
Avg qvals: -1.44396
Episode 78:
Reward: -20.0
Steps: 1451
Avg loss: 0.01795
Avg qvals: -1.55679
Episode 79:
Reward: -21.0
Steps: 1272
Avg loss: 0.01777
Avg qvals: -1.46793
Model saved.
Episode 80:
Reward: -21.0
Steps: 1048
Avg loss: 0.01723
Avg qv

Episode 136:
Reward: -21.0
Steps: 1022
Avg loss: 0.01722
Avg qvals: -1.34071
Episode 137:
Reward: -21.0
Steps: 1011
Avg loss: 0.01679
Avg qvals: -1.42551
Episode 138:
Reward: -21.0
Steps: 1026
Avg loss: 0.01667
Avg qvals: -1.53372
Episode 139:
Reward: -21.0
Steps: 1030
Avg loss: 0.01776
Avg qvals: -1.75186
Model saved.
Episode 140:
Reward: -21.0
Steps: 1019
Avg loss: 0.01871
Avg qvals: -1.58880
Episode 141:
Reward: -21.0
Steps: 1171
Avg loss: 0.01793
Avg qvals: -1.68215
Episode 142:
Reward: -21.0
Steps: 1105
Avg loss: 0.01841
Avg qvals: -1.57791
Episode 143:
Reward: -21.0
Steps: 1342
Avg loss: 0.01846
Avg qvals: -1.57415
Episode 144:
Reward: -20.0
Steps: 1250
Avg loss: 0.01700
Avg qvals: -1.62093
Episode 145:
Reward: -21.0
Steps: 1136
Avg loss: 0.01826
Avg qvals: -1.60971
Episode 146:
Reward: -20.0
Steps: 1171
Avg loss: 0.01764
Avg qvals: -1.79169
Episode 147:
Reward: -21.0
Steps: 1218
Avg loss: 0.01914
Avg qvals: -1.81584
Episode 148:
Reward: -18.0
Steps: 1385
Avg loss: 0.01733
Avg qv

Episode 203:
Reward: -20.0
Steps: 1112
Avg loss: 0.01763
Avg qvals: -1.47908
Episode 204:
Reward: -20.0
Steps: 1128
Avg loss: 0.01588
Avg qvals: -1.54764
Episode 205:
Reward: -21.0
Steps: 1195
Avg loss: 0.01782
Avg qvals: -1.65002
Episode 206:
Reward: -21.0
Steps: 1092
Avg loss: 0.01706
Avg qvals: -1.67360
Episode 207:
Reward: -21.0
Steps: 1088
Avg loss: 0.01744
Avg qvals: -1.56666
Episode 208:
Reward: -20.0
Steps: 1219
Avg loss: 0.01744
Avg qvals: -1.56931
Episode 209:
Reward: -20.0
Steps: 1139
Avg loss: 0.01779
Avg qvals: -1.71384
Model saved.
Episode 210:
Reward: -21.0
Steps: 1103
Avg loss: 0.01608
Avg qvals: -1.48564
Episode 211:
Reward: -20.0
Steps: 1121
Avg loss: 0.01656
Avg qvals: -1.63207
Episode 212:
Reward: -21.0
Steps: 1094
Avg loss: 0.01877
Avg qvals: -1.72821
Episode 213:
Reward: -20.0
Steps: 1127
Avg loss: 0.01676
Avg qvals: -1.50630
Episode 214:
Reward: -21.0
Steps: 1174
Avg loss: 0.01710
Avg qvals: -1.44992
Episode 215:
Reward: -21.0
Steps: 1126
Avg loss: 0.01888
Avg qv

Episode 270:
Reward: -20.0
Steps: 1122
Avg loss: 0.01805
Avg qvals: -1.47548
Episode 271:
Reward: -20.0
Steps: 1387
Avg loss: 0.01758
Avg qvals: -1.53755
Episode 272:
Reward: -20.0
Steps: 1360
Avg loss: 0.01753
Avg qvals: -1.49485
Episode 273:
Reward: -20.0
Steps: 1352
Avg loss: 0.01745
Avg qvals: -1.53560
Episode 274:
Reward: -20.0
Steps: 1119
Avg loss: 0.01843
Avg qvals: -1.68706
Episode 275:
Reward: -21.0
Steps: 1097
Avg loss: 0.01811
Avg qvals: -1.76823
Episode 276:
Reward: -20.0
Steps: 1175
Avg loss: 0.01735
Avg qvals: -1.82115
Episode 277:
Reward: -19.0
Steps: 1261
Avg loss: 0.01776
Avg qvals: -1.51873
Episode 278:
Reward: -20.0
Steps: 1259
Avg loss: 0.01806
Avg qvals: -1.70976
Episode 279:
Reward: -21.0
Steps: 1107
Avg loss: 0.01796
Avg qvals: -1.50011
Model saved.
Episode 280:
Reward: -20.0
Steps: 1277
Avg loss: 0.01760
Avg qvals: -1.60981
Episode 281:
Reward: -18.0
Steps: 1359
Avg loss: 0.01765
Avg qvals: -1.72821
Episode 282:
Reward: -20.0
Steps: 1397
Avg loss: 0.01802
Avg qv

...

Episode 1280:
Reward: -21.0
Steps: 1145
Avg loss: 0.01978
Avg qvals: -1.78576
Episode 1281:
Reward: -21.0
Steps: 1014
Avg loss: 0.01893
Avg qvals: -1.64835
Episode 1282:
Reward: -21.0
Steps: 1062
Avg loss: 0.01806
Avg qvals: -1.76483
Episode 1283:
Reward: -21.0
Steps: 1049
Avg loss: 0.01805
Avg qvals: -1.50967
Episode 1284:
Reward: -18.0
Steps: 1339
Avg loss: 0.01904
Avg qvals: -1.93343
Episode 1285:
Reward: -20.0
Steps: 1161
Avg loss: 0.01750
Avg qvals: -1.54005
Episode 1286:
Reward: -21.0
Steps: 1085
Avg loss: 0.01711
Avg qvals: -1.46661
Episode 1287:
Reward: -21.0
Steps: 1094
Avg loss: 0.02015
Avg qvals: -1.85831
Episode 1288:
Reward: -21.0
Steps: 1017
Avg loss: 0.01912
Avg qvals: -1.62444
Episode 1289:
Reward: -18.0
Steps: 1401
Avg loss: 0.01786
Avg qvals: -1.73574
Model saved.
Episode 1290:
Reward: -21.0
Steps: 1127
Avg loss: 0.01768
Avg qvals: -1.57248
Episode 1291:
Reward: -20.0
Steps: 1187
Avg loss: 0.01981
Avg qvals: -1.80797
Episode 1292:
Reward: -21.0
Steps: 1051
Avg loss: 0

Episode 1347:
Reward: -20.0
Steps: 1164
Avg loss: 0.01865
Avg qvals: -1.68758
Episode 1348:
Reward: -20.0
Steps: 1141
Avg loss: 0.01820
Avg qvals: -1.59451
Episode 1349:
Reward: -20.0
Steps: 1150
Avg loss: 0.01889
Avg qvals: -1.83157
Model saved.
Episode 1350:
Reward: -21.0
Steps: 1024
Avg loss: 0.01750
Avg qvals: -1.51452
Episode 1351:
Reward: -21.0
Steps: 1005
Avg loss: 0.01875
Avg qvals: -1.73966
Episode 1352:
Reward: -21.0
Steps: 1164
Avg loss: 0.01827
Avg qvals: -1.46176
Episode 1353:
Reward: -20.0
Steps: 1107
Avg loss: 0.01815
Avg qvals: -1.65653
Episode 1354:
Reward: -21.0
Steps: 1083
Avg loss: 0.01901
Avg qvals: -1.78708
Episode 1355:
Reward: -21.0
Steps: 1060
Avg loss: 0.01831
Avg qvals: -1.82218
Episode 1356:
Reward: -20.0
Steps: 1127
Avg loss: 0.01969
Avg qvals: -1.85580
Episode 1357:
Reward: -21.0
Steps: 1017
Avg loss: 0.01909
Avg qvals: -1.91006
Episode 1358:
Reward: -21.0
Steps: 1043
Avg loss: 0.01797
Avg qvals: -1.80359
Episode 1359:
Reward: -21.0
Steps: 1065
Avg loss: 0

Episode 1414:
Reward: -20.0
Steps: 1143
Avg loss: 0.01914
Avg qvals: -1.75235
Episode 1415:
Reward: -20.0
Steps: 1159
Avg loss: 0.01798
Avg qvals: -1.65576
Episode 1416:
Reward: -21.0
Steps: 1092
Avg loss: 0.01962
Avg qvals: -1.87114
Episode 1417:
Reward: -21.0
Steps: 1011
Avg loss: 0.02019
Avg qvals: -1.81749
Episode 1418:
Reward: -21.0
Steps: 1028
Avg loss: 0.01809
Avg qvals: -1.66692
Episode 1419:
Reward: -21.0
Steps: 1136
Avg loss: 0.01787
Avg qvals: -1.62437
Model saved.
Episode 1420:
Reward: -21.0
Steps: 1049
Avg loss: 0.01997
Avg qvals: -1.74695
Episode 1421:
Reward: -21.0
Steps: 1026
Avg loss: 0.02045
Avg qvals: -1.83599
Episode 1422:
Reward: -21.0
Steps: 1017
Avg loss: 0.01780
Avg qvals: -1.55712
Episode 1423:
Reward: -21.0
Steps: 1178
Avg loss: 0.01861
Avg qvals: -1.73912
Episode 1424:
Reward: -21.0
Steps: 1009
Avg loss: 0.01852
Avg qvals: -1.75277
Episode 1425:
Reward: -20.0
Steps: 1135
Avg loss: 0.01865
Avg qvals: -1.89768
Episode 1426:
Reward: -20.0
Steps: 1278
Avg loss: 0

Episode 1481:
Reward: -21.0
Steps: 1019
Avg loss: 0.01914
Avg qvals: -1.76436
Episode 1482:
Reward: -21.0
Steps: 1041
Avg loss: 0.01741
Avg qvals: -1.53654
Episode 1483:
Reward: -21.0
Steps: 1004
Avg loss: 0.01851
Avg qvals: -1.67582
Episode 1484:
Reward: -21.0
Steps: 1021
Avg loss: 0.01854
Avg qvals: -1.56474
Episode 1485:
Reward: -21.0
Steps: 1020
Avg loss: 0.01874
Avg qvals: -2.02387
Episode 1486:
Reward: -21.0
Steps: 1016
Avg loss: 0.01871
Avg qvals: -1.69595
Episode 1487:
Reward: -21.0
Steps: 1020
Avg loss: 0.01778
Avg qvals: -1.52573
Episode 1488:
Reward: -21.0
Steps: 1015
Avg loss: 0.01822
Avg qvals: -1.61536
Episode 1489:
Reward: -21.0
Steps: 1023
Avg loss: 0.01770
Avg qvals: -1.68665
Model saved.
Episode 1490:
Reward: -21.0
Steps: 1012
Avg loss: 0.01911
Avg qvals: -1.66508
Episode 1491:
Reward: -21.0
Steps: 1024
Avg loss: 0.01904
Avg qvals: -1.75541
Episode 1492:
Reward: -21.0
Steps: 1029
Avg loss: 0.01909
Avg qvals: -1.54582
Episode 1493:
Reward: -21.0
Steps: 1020
Avg loss: 0

KeyboardInterrupt: 

## Manually Go Through One Iteration

In [ ]:
import numpy as np
import torch

import gym
from src import agent, dqn, train, utils

#### Create Gym Environment

In [ ]:
env = gym.envs.make('Pong-v4')

In [ ]:
init_obs = env.reset()

for i in range(20000):
    a = random.randrange(env.action_space.n)
    _, _, done, _ = env.step(a)
    time.sleep(.01)
    env.render()
    
    if done:
        env.reset()

In [ ]:
init_obs = env.reset()
init_obs.shape

In [ ]:
s1, r, done, _ = env.step(0)

In [ ]:
s1.shape

#### Initialize DQN objects

In [ ]:
replay_mem_size = int(1e6)
mini_batch_size = 32

In [ ]:
agt = agent.DQNAgent()
replay_memory = utils.ReplayMemory(replay_mem_size, mini_batch_size)
obs_history = utils.ObsHistory()

#### Begin new episode

In [ ]:
obs_init = env.reset()  # reset environment to start new episode
obs_history.reset(obs_init)  # reset observations for new episode
done = False

#### Choose action

In [ ]:
phi = obs_history.phi
a = agt.act(phi)
obs, rew, done, _ = env.step(a)
obs_history.store(obs)

#### Store Transition

In [ ]:
phi_1 = obs_history.phi
replay_memory.store((phi, a, rew, phi_1, done))

#### Gradient Step

[x] Dummy transitions function for testing.

[x] Make `r` reward vector from transitions.

[x] Make $\max_{a}Q(s', a')$ vector from transitions.

[x] Make `y` target vector from transitions.

In [ ]:
from collections import namedtuple
import torch
from torch import nn
import numpy as np

Transition = namedtuple('Transition', 
                        ['phi', 'a', 'r', 'phi_1', 'done'])

In [ ]:
def dummy_transitions(n):
    transitions = []
    
    for i in range(n):
        phi = torch.empty(4, 84, 84).random_(0, 255)
        phi_1 = torch.empty(4, 84, 84).random_(0, 255)
        a = np.random.randint(0, 6)
        r = np.random.randint(0, 2)
        done = False if np.random.randint(0, 2) == 0 else True
        
        transitions.append(Transition(phi, a, r, phi_1, done))
    
    return transitions

In [ ]:
transitions = dummy_transitions(3)
    
phi, a, r, phi_1, done = zip(*transitions)

In [ ]:
def make_y(transitions, agt):
    y = []
    
    for tr in transitions:
        if tr.done:
            y.append(tr.r)
        else:
            x = tr.phi.unsqueeze(0)
            y.append(tr.r + .99 * agt.get_best_values(x).item())
            
    return torch.Tensor(y)

In [ ]:
y = make_y(transitions, agt)

In [ ]:
def get_max_vals(transitions, agt):
    phis = []

    for tr in transitions:
        phis.append(tr.phi)

    x = torch.stack(phis)
    return agt.get_best_values(x)

In [ ]:
qmax = get_max_vals(transitions, agt)

In [ ]:
loss = nn.MSELoss(reduction='mean')

In [ ]:
loss(y, qmax)

In [ ]:
def mini_batch_loss(transitions, agt):
    y = make_y(transitions, agt)
    qmax = get_max_vals(transitions, agt)
    
    loss = nn.MSELoss(reduction='mean')
    return loss(y, qmax)

In [ ]:
mb_loss = mini_batch_loss(transitions, agt)

#### Optimizer and gradient step

In [ ]:
from torch import optim

optimizer = optim.RMSprop(agt.qnet.parameters())

In [ ]:
optimizer.zero_grad()

In [ ]:
mb_loss = mini_batch_loss(transitions, agt)
mb_loss

In [ ]:
mb_loss.backward()

In [ ]:
optimizer.step()

### Test random agent on Pong

In [ ]:
s0 = env.reset()

In [ ]:
obss = []
obss.append(env.reset())

for _ in range(1000):
    a = np.random.choice(env.action_space.n)
    obs, rew, done, _ = env.step(a)
    obss.append(obs)
    
    if done:
        obs = env.reset()

In [ ]:
env.action_space.n